In [1]:
from src.llm_plan.environment import Environment

env = Environment("./environments/static/TwoAgentsVault.json")
env.plan

[['key_holder.pddl', 'small_robot.pddl'], ['orchestrator.pddl']]

In [2]:
# Create the plan
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

model = GPT_4o()
planner = Planner(env, model)

In [3]:
# Plan
responses = planner.plan()

----- key_holder->pddl -----

----- System Prompt -----
You are an expert with PDDL problems (Planning Domain Definition Language). You always provide a PDDL domain and a PDDL problem file to solve the task. You always enclose the pddl domain between <domain></domain> tags, and the pddl problem between <problem></problem> tags.

----- Prompt -----
Your name is key_holder. You are in an environment with the following public information:
['There is a vault in the environment.', 'The vault is closed.', 'The vault requires a key to be opened.', 'The entrance of the vault is small.']
You have the following knowledge:
['I have a key', 'I am a big robot']
This is the global goal to solve:
Open the vault and grab the object inside.
Think step by step and and provide a PDDL domain and a PDDL problem file to solve the task.
If you miss some information, do not make assumptions, just give a plan that concerns the information you have.
Enclose the pddl domain between <domain></domain> tags, and th

In [4]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_orchestrator"
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [5]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [6]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected



In [7]:
# Let's try to refine the plan syntax
from src.llm_plan.hypervisor import HypervisorDeepThinkPDDL, HypervisorSyntaxPDDL

num_attempts = 10

for i in range(num_attempts):
    
    # General validator
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = HypervisorDeepThinkPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
        f.write(domain)
        
    # Launch the solver
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / "refined_sas_plan",
        BASE_FOLDER / "refined_domain.pddl",
        BASE_FOLDER / "refined_problem.pddl",
    ]

    with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / 'refined_domain.pddl'} \
    {BASE_FOLDER / 'refined_problem.pddl'} \
    {BASE_FOLDER / 'refined_sas_plan'}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)

    # This part won't be printed at test time
    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # Syntax validator
    print(f"Validation attempt {i+1}/{num_attempts}")
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
    }

    deep_think = HypervisorSyntaxPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attempt 1/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attempt 2/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attempt 3/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attempt 4/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attempt 5/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attem

In [8]:
# Final plan run and validation
with open(BASE_FOLDER / "refined_problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "refined_domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "refined_sas_plan",
    BASE_FOLDER / "refined_domain.pddl",
    BASE_FOLDER / "refined_problem.pddl",
]

with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'refined_domain.pddl'} \
{BASE_FOLDER / 'refined_problem.pddl'} \
{BASE_FOLDER / 'refined_sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The plan is valid.
[stdout] Checking plan: tmp/TwoAgentsVault/refined_sas_plan
Plan executed successfully - checking goal
Success: Plan valid
Value: 2.000000


